In [1]:
import pandas as pd
from pandas import DataFrame
from sklearn.utils import shuffle

Will perform the following:

1. Merge text from different columns together. The merged data frame will have 3 columns:

    ```
    ['LABEL', 'TEXT', 'SOURCE']
    ```

2. Minimal normalization of converting to lower case and stripping whitespace
3. Shuffle the merged data

Let's not be agressive with normalization. Only upper case to lower case.

Some observations when considering normalization:

* 2-Amino-4,6-pteridinediol
* Ser/Plas
* [Moles/volume]
* {Setting}
* Pulmonary vein - right upper 
* Multisection^W contrast IV
* Bacteria identified^^^2
* ^Patient
* Blood flow.diastole.max
* Upper extremity>Upper arm
* A+B Ag

In [2]:
def rename_columns(df, label_col, text_col):
    """

    Renames the selected label column to 'LABEL' and
    the text column to 'TEXT'. Adds a new column 'SOURCE'
    indicating the source.

    df: DataFrame
    label_col: string, the label column
    text_col: string, the text column
    return: a new data frame with columns renamed

    """
    df = df.rename(index=str, columns={
        label_col: 'LABEL',
        text_col: 'TEXT'
    })
    df['SOURCE'] = text_col
    return df

In [3]:
def normalize(df):
    """

    Adds a normalized text column. Normalization is lightweight
    only converting to lower case and stripping whitespace.

    df: DataFrame
    return: void, adding a new column

    """
    df['TEXT_NORMED'] = df['TEXT'].apply(lambda x: str(x).lower().strip())

In [4]:
def merge(df_dicts):
    """
    dataframes: list of data frame dict {df, label_col, text_col}
    return: the merged, normalized, shuffled data frame
    """
    # Merge data frames
    dfs = []
    for df_dict in df_dicts:
        df = df_dict['df']
        label_col = df_dict['label_col']
        text_col = df_dict['text_col']
        df = rename_columns(df, label_col, text_col)
        dfs.append(df)
    df_merged = pd.concat(dfs, ignore_index=True)
    # Drop duplicates
    df_merged = df_merged.drop_duplicates()
    # Normalize text
    normalize(df_merged)
    # Drop rows that has null and empty text
    # Pandas gotcha, missing values internally as NaN, 'is not None' won't work
    df_merged = df_merged[df_merged['TEXT'].notnull()]
    df_merged = df_merged[df_merged['TEXT_NORMED'].map(lambda text: len(text) > 0)]
    # Shuffle
    df_merged = shuffle(df_merged)
    return df_merged

### Merge all the candidate columns

In [5]:
df_component = pd.read_csv('data/loinc-component.csv')
df_short_name = pd.read_csv('data/loinc-short-name.csv')
df_long_name = pd.read_csv('data/loinc-long-name.csv')
df_system = pd.read_csv('data/loinc-system.csv')
df_class = pd.read_csv('data/loinc-class.csv')
df_related = pd.read_csv('data/loinc-related-name.csv')

In [6]:
df_dicts = [
    {
        'df': df_component,
        'label_col': 'LOINC_NUM',
        'text_col': 'COMPONENT'
    }, {
        'df': df_short_name,
        'label_col': 'LOINC_NUM',
        'text_col': 'SHORTNAME'
    }, {
        'df': df_long_name,
        'label_col': 'LOINC_NUM',
        'text_col': 'LONG_COMMON_NAME'
    }, {
        'df': df_system,
        'label_col': 'LOINC_NUM',
        'text_col': 'SYSTEM'
    }, {
        'df': df_class,
        'label_col': 'LOINC_NUM',
        'text_col': 'CLASS'
    }, {
        'df': df_related,
        'label_col': 'LOINC_NUM',
        'text_col': 'RELATED_NAME'
    }
]

In [7]:
df_merged_all = merge(df_dicts)

In [8]:
df_merged_all.sample(10)

,LABEL,TEXT,SOURCE,TEXT_NORMED
2100893,81274-3,Desmethyl,RELATED_NAME,desmethyl
44937,50943-0,Reasons for assessment (full),COMPONENT,reasons for assessment (full)
1970718,7509-3,Aby,RELATED_NAME,aby
1540634,54315-7,Quan,RELATED_NAME,quan
106098,30755-3,T-spine XR port AP 1V,SHORTNAME,t-spine xr port ap 1v
973852,31785-9,Spinal Fluid,RELATED_NAME,spinal fluid
1215943,41329-4,Point in time,RELATED_NAME,point in time
2097226,8106-7,CELL MARKERS,RELATED_NAME,cell markers
1438099,5019-5,3 Self-Sustaining Sequence Replication,RELATED_NAME,3 self-sustaining sequence replication
868090,27377-1,Aspergilla,RELATED_NAME,aspergilla


In [9]:
df_merged_all.shape

(2198746, 4)

### Merge only the columns of long names and short names

In [10]:
df_dicts = [
    {
        'df': df_short_name,
        'label_col': 'LOINC_NUM',
        'text_col': 'SHORTNAME'
    }, {
        'df': df_long_name,
        'label_col': 'LOINC_NUM',
        'text_col': 'LONG_COMMON_NAME'
    }
]

In [11]:
df_merged_names = merge(df_dicts)

In [12]:
df_merged_names.sample(5)

,LABEL,TEXT,SOURCE,TEXT_NORMED
100260,25441-7,Homocystine [Moles/time] in 24 hour Urine,LONG_COMMON_NAME,homocystine [moles/time] in 24 hour urine
117102,40780-9,Mexiletine [Mass/volume] in Serum or Plasma --...,LONG_COMMON_NAME,mexiletine [mass/volume] in serum or plasma --...
86757,12950-2,Deprecated Triglyceride [Mass/volume] in Body ...,LONG_COMMON_NAME,deprecated triglyceride [mass/volume] in body ...
31286,38571-6,Barium Air-mCnc,SHORTNAME,barium air-mcnc
9882,19792-1,Exam extent landmark UGI EGD,SHORTNAME,exam extent landmark ugi egd


In [13]:
df_merged_names.shape

(157577, 4)

### Verify

In [14]:
df_merged_all[df_merged_all['LABEL'] == '80659-6']

,LABEL,TEXT,SOURCE,TEXT_NORMED
2088251,80659-6,Point in time,RELATED_NAME,point in time
2088261,80659-6,Whole blood,RELATED_NAME,whole blood
410791,80659-6,COAG,CLASS,coag
77283,80659-6,Activated clotting time,COMPONENT,activated clotting time
2088252,80659-6,QNT,RELATED_NAME,qnt
2088257,80659-6,Rate,RELATED_NAME,rate
2088260,80659-6,WB,RELATED_NAME,wb
2088250,80659-6,Coagulation time,RELATED_NAME,coagulation time
2088248,80659-6,Coagulation activated,RELATED_NAME,coagulation activated
2088242,80659-6,Activ,RELATED_NAME,activ


In [15]:
df_merged_names[df_merged_names['LABEL'] == '80659-6']

,LABEL,TEXT,SOURCE,TEXT_NORMED
77283,80659-6,ACT Bld Kaolin induc,SHORTNAME,act bld kaolin induc
160660,80659-6,Activated clotting time (ACT) of Blood induced...,LONG_COMMON_NAME,activated clotting time (act) of blood induced...


In [16]:
# Missing short name represented as NaN
df_short_name[df_short_name['LOINC_NUM'] == '72156-3']

,LOINC_NUM,SHORTNAME
68050,72156-3,NaN


In [17]:
df_merged_names[df_merged_names['LABEL'] == '72156-3']

,LABEL,TEXT,SOURCE,TEXT_NORMED
151427,72156-3,Rwandan maternal screening panel [RHEA],LONG_COMMON_NAME,rwandan maternal screening panel [rhea]


### Save

In [18]:
df_merged_all.to_csv('data/loinc-labeled-text-all.csv', index=False)
df_merged_names.to_csv('data/loinc-labeled-text-names.csv', index=False)